# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 37 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.01524


extracting tarball to tmp_2303.01524...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.01528


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.01524/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2303.01854...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.01857


extracting tarball to tmp_2303.01857...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.01857/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.01528-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.01528) | **The PAndAS View of the Andromeda Satellite System. IV Global properties**  |
|| Amandine Doliva-Dolinsky, et al. -- incl., <mark>Geraint F. Lewis</mark> |
|*Appeared on*| *2023-03-06*|
|*Comments*| *Submitted to ApJ - 12 pages, 6 figures, 2 tables*|
|**Abstract**| We build a statistical framework to infer the global properties of the satellite system of the Andromeda galaxy (M31) from the properties of individual dwarf galaxies located in the Pan-Andromeda Archaelogical Survey (PAndAS) and the previously determined completeness of the survey. Using forward modeling, we infer the slope of the luminosity function of the satellite system, the slope of its spatial density distribution, and the size-luminosity relation followed by the dwarf galaxies. We find that the slope of the luminosity function is $\beta=-1.5\pm0.1$. Combined with the spatial density profile, it implies that, when accounting for survey incompleteness, M31 hosts $92_{-26}^{+19}$ dwarf galaxies with $M_\textrm{V}<-5.5$ and a sky-projected distance from M31 between 30 and 300kpc. We conclude that many faint or distant dwarf galaxies remain to be discovered around Andromeda, especially outside the PAndAS footprint. Finally, we use our model to test if the higher number of satellites situated in the hemisphere facing the Milky Way could be explained simply by the detection limits of dwarf galaxy searches. We rule this out at $>99.9\%$ confidence and conclude that this anisotropy is an intrinsic feature of the M31 satellite system. The statistical framework we present here is a powerful tool to robustly constrain the properties of a satellite system and compare those across hosts, especially considering the upcoming start of the Euclid or Rubin large photometric surveys that are expected to uncover a large number of dwarf galaxies in the Local Volume. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.01524-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.01524) | **Where are the Cascades from Blazar Jets? An Emerging Tension in the  $γ$-ray sky**  |
|| Carlos Blanco, <mark>Oindrila Ghosh</mark>, Sunniva Jacobsen, Tim Linden |
|*Appeared on*| *2023-03-06*|
|*Comments*| **|
|**Abstract**| Blazars are among the most powerful accelerators and are expected to produce a bright TeV $\gamma$-ray flux. However, TeV $\gamma$-rays are attenuated by interactions with intergalactic radiation before reaching Earth. These interactions produce cascades that transfer the TeV power into the GeV band, powering both extended halos around bright sources and a large contribution to the isotropic $\gamma$-ray background (IGRB). Using state-of-the-art blazar models and recent IGRB measurements, we rule out models where blazars effectively transfer their TeV power into GeV $\gamma$-rays. Three possible solutions include: (1) strong spectral cuts on bright blazars, which are at odds with local blazar observations, (2) collective plasma effects that can prevent the development of blazar cascades, the effectiveness of which is debated, (3) an increase in the effective $\gamma$-ray opacity from axion-like particles. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.01637-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.01637) | **Multimass modelling of Milky Way globular clusters -- I. Implications on  their stellar initial mass function above 1 M$_{\odot}$**  |
|| Nolan Dickson, et al. -- incl., <mark>Peter Smith</mark> |
|*Appeared on*| *2023-03-06*|
|*Comments*| *21 pages, 10 figures, submitted to MNRAS. Comments welcome*|
|**Abstract**| The distribution of stars and stellar remnants (white dwarfs, neutron stars, black holes) within globular clusters holds clues about their formation and long-term evolution, with important implications for their initial mass function (IMF) and the formation of black hole mergers. In this work, we present best-fitting multimass models for 37 Milky Way globular clusters, which were inferred from various datasets, including proper motions from Gaia EDR3 and HST, line-of-sight velocities from ground-based spectroscopy and deep stellar mass functions from HST. We use metallicity dependent stellar evolution recipes to obtain present-day mass functions of stars and remnants from the IMF. By dynamically probing the present-day mass function of all objects in a cluster, including the mass distribution of remnants, these models allow us to explore in detail the stellar (initial) mass functions of a large sample of Milky Way GCs. We show that, while the low-mass mass function slopes are strongly dependent on the dynamical age of the clusters, the high-mass slope ($\alpha_3; m > 1 M_\odot$) is not, indicating that the mass function in this regime has generally been less affected by dynamical mass loss. Examination of this high-mass mass function slope suggests an IMF in this mass regime consistent with a Salpeter IMF is required to reproduce the observations. This high-mass IMF is incompatible with a top-heavy IMF, as has been proposed recently. Finally, no significant correlation is found between the high-mass IMF slope and cluster metallicity. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.01658-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.01658) | **ALMA Lensing Cluster Survey: Deep 1.2 mm Number Counts and Infrared  Luminosity Functions at $z\simeq1-8$**  |
|| Seiji Fujimoto, et al. -- incl., <mark>Yoshihiro Ueda</mark> |
|*Appeared on*| *2023-03-06*|
|*Comments*| *Submitted to ApJS, 34 pages, 8 Tables, and 18 Figures (+38 pages, 5 Tables, and 6 Figures in Appendix)*|
|**Abstract**| We present a statistical study of 180 dust continuum sources identified in 33 massive cluster fields by the ALMA Lensing Cluster Survey (ALCS) over a total of 133 arcmin$^{2}$ area, homogeneously observed at 1.2 mm. ALCS enables us to detect extremely faint mm sources by lensing magnification, including near-infrared (NIR) dark objects showing no counterparts in existing {\it Hubble Space Telescope} and {\it Spitzer} images. The dust continuum sources belong to a blind sample ($N=141$) with S/N $\gtrsim$ 5.0 (a purity of $>$ 0.99) or a secondary sample ($N=39$) with S/N= $4.0-5.0$ screened by priors. With the blind sample, we securely derive 1.2-mm number counts down to $\sim7$ $\mu$Jy, and find that the total integrated 1.2mm flux is 20.7$^{+8.5}_{-6.5}$ Jy deg$^{-2}$, resolving $\simeq$ 80 % of the cosmic infrared background light. The resolved fraction varies by a factor of $0.6-1.1$ due to the completeness correction depending on the spatial size of the mm emission. We also derive infrared (IR) luminosity functions (LFs) at $z=0.6-7.5$ with the $1/V_{\rm max}$ method, finding the redshift evolution of IR LFs characterized by positive luminosity and negative density evolution. The total (=UV+IR) cosmic star-formation rate density (SFRD) at $z>4$ is estimated to be $161^{+25}_{-21}$ % of the established measurements, which were almost exclusively based on optical$-$NIR surveys. Although our general understanding of the cosmic SFRD is unlikely to change beyond a factor of 2, these results add to the weight of evidence for an additional ($\approx 60$ %) SFRD component contributed by the faint-mm population, including NIR dark objects. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.01753-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.01753) | **Asteroseismic Determination of Stellar Rotation: On Synchronization in  the Close Eclipsing Binaries AB Cas and OO Dra**  |
|| <mark>Seung-Lee Kim</mark> |
|*Appeared on*| *2023-03-06*|
|*Comments*| *17 pages, accepted for publication in ApJ*|
|**Abstract**| A star's rotation rate is difficult to estimate without surface inhomogeneities such as dark or bright spots. This paper presents asteroseismic results to determine the rotation rates of $\delta$ Sct-type pulsating primary stars in two eclipsing binary systems, AB Cas and OO Dra. After removing the binarity-induced light variations from the archival TESS data and carefully examining the combination frequencies, we identified 12 independent frequencies for AB Cas and 11 frequencies for OO Dra, with amplitudes higher than $\sim$0.3 mmag, as $\delta$ Sct-type pulsation frequencies excited in each primary star. The theoretical frequencies for seismic analysis were obtained by fully considering the rotation effects. Grid fitting for various stellar properties, such as mass, radius, metallicity, and rotation rate, yielded the best solution for which theoretical frequencies and stellar parameters agreed well with the observations. The rotation rate of the AB Cas primary was tightly constrained to 0.81 $\pm$ 0.01 day$^{-1}$ ($f_{\rm rot} / f_{\rm orb}$ = 1.11$^{+0.01}_{-0.02}$), which is slightly faster than the synchronized rotation. In contrast, the rotation rate of 0.63 $\pm$ 0.01 day$^{-1}$ for the OO Dra primary is lower than the synchronous value of approximately 0.81 day$^{-1}$. Subsynchronous rotation is uncommon in short-period binaries, and its physical mechanism is not yet well understood. Our results show that asteroseismology can be used to precisely measure the rotation rate of fast-rotating $\delta$ Sct stars and thus provide a valuable constraint on rotation-orbit synchronization in close binary systems. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.01854-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.01854) | **Chemical models of adenine precursors cyanamide and carbodiimide in the  interstellar medium**  |
|| <mark>Xia Zhang</mark>, et al. -- incl., <mark>Runxia Li</mark> |
|*Appeared on*| *2023-03-06*|
|*Comments*| *13 pages, 6 figures, 7 tables, accepted for publication in MNRAS*|
|**Abstract**| Cyanamide (NH2CN) and its isomer, carbodiimide (HNCNH), may form adenine in the interstellar medium (ISM) via a series of reactions. Therefore, they are considered key prebiotic molecules in the study of the origin of life. We used the three-phase NAUTILUS chemical code, which includes the gas, the dust surface, and the icy mantle, to investigate the formation and destruction of cyanamide and carbodiimide. We added over 200 new chemical reactions of the two isomers and related species, and established a relatively complete network. We applied cold core, hot corino/core and shock models to simulate the different physical environments, and found that the two isomers are mainly produced by the free radical reactions on grain surfaces. Our simulated results suggest that cyanamide and carbodiimide molecules come from surface chemistry at early evolutionary stages. Then they are released back to the gas phase, either by thermal process (in hot cores, hot corinos) or shock-induced desorption (in shock regions).We speculate that it is an inefficient route to form a tautomer of adenine by starting from molecules cyanoacetylene (C3NH), cyanamide and carbodiimide in ISM. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.01857-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.01857) | **The Fast X-ray Transient XRT 210423 and its Host Galaxy**  |
|| D. Eappachen, et al. -- incl., <mark>M.A.P. Torres</mark> |
|*Appeared on*| *2023-03-06*|
|*Comments*| *14 pages, 6 figures. Accepted for publication in ApJ*|
|**Abstract**| Fast X-ray Transients (FXTs) are X-ray flares with a duration ranging from a few hundred seconds to a few hours. Possible origins include the tidal disruption of a white dwarf by an intermediate-mass black hole, a supernova shock breakout, and a binary neutron star merger. We present the X-ray light curve and spectrum, and deep optical imaging of the FXT XRT 210423, which has been suggested to be powered by a magnetar produced in a binary neutron star merger. Our Very Large Telescope and Gran Telescopio Canarias (GTC) observations began on May 6, 2021, thirteen days after the onset of the flare. No transient optical counterpart is found in the 1" (3$\sigma$) X-ray uncertainty region of the source to a depth $g_{s}$=27.0 AB mag. A candidate host lies within the 1" X-ray uncertainty region with a magnitude of 25.9 $\pm$ 0.1 in the GTC/HiPERCAM $g_s$-filter. Due to its faintness, it was not detected in other bands, precluding a photometric redshift determination. We detect two additional candidate host galaxies; one with $z_{\rm spec}=1.5082 \pm 0.0001$ and an offset of 4.2$\pm$1" (37$\pm$9 kpc) from the FXT and another one with $z_{\rm phot}=1.04^{+0.22}_{-0.14}$, at an offset of 3.6$\pm$1" (30$\pm$8 kpc). Based on the properties of all the prospective hosts we favour a binary neutron star merger, as previously suggested in the literature, as explanation for XRT 210423. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.01700-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.01700) | **Ejecta from the DART-produced active asteroid Dimorphos**  |
|| <mark>Jian-Yang Li</mark>, et al. |
|*Appeared on*| *2023-03-06*|
|*Comments*| *accepted by Nature*|
|**Abstract**| Some active asteroids have been proposed to be the result of impact events. Because active asteroids are generally discovered serendipitously only after their tail formation, the process of the impact ejecta evolving into a tail has never been directly observed. NASA's Double Asteroid Redirection Test (DART) mission, apart from having successfully changed the orbital period of Dimorphos, demonstrated the activation process of an asteroid from an impact under precisely known impact conditions. Here we report the observations of the DART impact ejecta with the Hubble Space Telescope (HST) from impact time T+15 minutes to T+18.5 days at spatial resolutions of ~2.1 km per pixel. Our observations reveal a complex evolution of ejecta, which is first dominated by the gravitational interaction between the Didymos binary system and the ejected dust and later by solar radiation pressure. The lowest-speed ejecta dispersed via a sustained tail that displayed a consistent morphology with previously observed asteroid tails thought to be produced by impact. The ejecta evolution following DART's controlled impact experiment thus provides a framework for understanding the fundamental mechanisms acting on asteroids disrupted by natural impact. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.01528.md
    + _build/html/tmp_2303.01528/./function_lumi_3D.png
    + _build/html/tmp_2303.01528/./mv_rh.png
    + _build/html/tmp_2303.01528/./DL_ani.png
    + _build/html/tmp_2303.01528/./observed_density_ani.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\fmmm}[1]{\mbox{#1}}$
$\newcommand{\scnd}{\mbox{\fmmm{"}\hskip-0.3em .}}$
$\newcommand{\scnp}{\mbox{\fmmm{"}}}$
$\newcommand{\mcnd}{\mbox{\fmmm{'}\hskip-0.3em .}}$
$\newcommand{\ud}{\mathrm{d}}$
$\newcommand{\ltsima}{\; \buildrel < \over \sim \;}$
$\newcommand{\simlt}{\lower.5ex\hbox{\ltsima}}$
$\newcommand{\gtsima}{\; \buildrel > \over \sim \;}$
$\newcommand{\simgt}{\lower.5ex\hbox{\gtsima}}$
$\newcommand{\erf}{\mathop{\rm erf}\nolimits}$
$\newcommand{\sech}{ \mathop{\rm sech}\nolimits}$
$\newcommand{\csch}{ \mathop{\rm csch}\nolimits}$
$\newcommand{\arcsinh}{\mathop{\rm arcsinh}\nolimits}$
$\newcommand{\arccosh}{\mathop{\rm arccosh}\nolimits}$
$\newcommand{\arctanh}{\mathop{\rm arctanh}\nolimits}$
$\newcommand{\arccoth}{\mathop{\rm arccoth}\nolimits}$
$\newcommand{\arcsech}{\mathop{\rm arcsech}\nolimits}$
$\newcommand{\arccsch}{\mathop{\rm arccsch}\nolimits}$
$\newcommand{\arccot}{\mathop{\rm arccot}\nolimits}$
$\newcommand{\arcsec}{\mathop{\rm arcsec}\nolimits}$
$\newcommand{\arccsc}{\mathop{\rm arccsc}\nolimits}$
$\newcommand{\ylm}{\mathop{\rm Y}_l^m\nolimits}$
$\newcommand{\ylmp}{\mathop{\rm Y}_{l'}^{m'}\nolimits}$
$\newcommand{\real}{\Re e}$
$\newcommand{\imag}{\Im m}$
$\newcommand{\km}{{\rm km}}$
$\newcommand{\kms}{{\rm km s^{-1}}}$
$\newcommand{\kmsdeg}{{\rm km s^{-1} deg^{-1}}}$
$\newcommand{\kmskpc}{{\rm km s^{-1} kpc^{-1}}}$
$\newcommand{\mas}{{\rm mas}}$
$\newcommand{\masyr}{{\rm mas/yr}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand{\kpcdeg}{{\rm kpc deg^{-1}}}$
$\newcommand{\mpc}{{\rm Mpc}}$
$\newcommand{\msun}{{\rm M_\odot}}$
$\newcommand{\lsun}{{\rm L_\odot}}$
$\newcommand{\rsun}{{\rm R_\odot}}$
$\newcommand{\pc}{{\rm pc}}$
$\newcommand{\cm}{{\rm cm}}$
$\newcommand{\yr}{{\rm yr}}$
$\newcommand{\au}{{\rm AU}}$
$\newcommand{\g}{{\rm g}}$
$\newcommand{\om}{\Omega_0}$
$\newcommand{\}{ca}$
$\newcommand{\}{r}$
$\newcommand{\}{magnitude}$
$\newcommand{\kr}{{\cal K}_r}$
$\newcommand{\kz}{{\cal K}_z}$
$\newcommand{\kzz}{{\cal K}_z(z)}$
$\newcommand{\mss}{{\rm M}_\odot \rm pc^{-2}}$
$\newcommand{\msss}{{\rm M}_\odot \rm pc^{-3}}$
$\newcommand{\Aa}{\; \buildrel \circ \over{\rm A}}$
$\newcommand{Å}{\; \buildrel \circ \over{\rm A}}$
$\newcommand{\yr}{{\rm yr}}$
$\newcommand{\CompactFigs}{0}$
$\newcommand{\UseFigs}{1}$
$\newcommand{\deg}{^\circ}$
$\newcommand{\degg}{\hbox{\null^\circ\hskip-3pt .}}$
$\newcommand{\sec}{\hbox{"\hskip-3pt .}}$
$\newcommand{\half}{{\scriptstyle{1\over2}}}$
$\newcommand{\s}{\ifmmode \widetilde \else \~\fi}$
$\newcommand{\=}{\overline}$
$\newcommand{\scre}{{\cal E}}$
$\newcommand{\}{spose}$
$\newcommand{\larrow}{\leftarrow}$
$\newcommand{\rarrow}{\rightarrow}$
$\newcommand{\llangle}{\langle\langle}$
$\newcommand{\rrangle}{\rangle\rangle}$
$\newcommand{\etal}{{\it et al. }}$
$\newcommand{\cf}{{\it cf. }}$
$\newcommand{\eg}{{e.g., }}$
$\newcommand{\ie}{{ i.e., }}$
$\newcommand{\lta}{\mathrel{\spose{\lower 3pt\hbox{\mathchar"218}}$
$     \raise 2.0pt\hbox{\mathchar"13C}}}$
$\newcommand{\gta}{\mathrel{\spose{\lower 3pt\hbox{\mathchar"218}}$
$     \raise 2.0pt\hbox{\mathchar"13E}}}$
$\newcommand{\Dt}{\spose{\raise 1.5ex\hbox{\hskip3pt\mathchar"201}}}$
$\newcommand{\dt}{\spose{\raise 1.0ex\hbox{\hskip2pt\mathchar"201}}}$
$\newcommand{\del}{\nabla}$
$\newcommand{\delv}{\bb\nabla}$
$\newcommand{\r}{{\rm r^{1/4}}}$
$\newcommand{\jla}{J_{\lambda}}$
$\newcommand{\jmu}{J_{\mu}}$
$\newcommand{\jnu}{J_{\nu}}$
$\newcommand{\pomega}{\varpi}$
$\newcommand{\sigla}{\sigma_{\lambda}}$
$\newcommand{\sigmu}{\sigma_{\mu}}$
$\newcommand{\signu}{\sigma_{\nu}}$
$\newcommand{\dotsfill}{\leaders\hbox to 1em{\hss.\hss}\hfill}$
$\newcommand{\sun}{\odot}$
$\newcommand{\earth}{\oplus}$
$\newcommand{\Gyr}{{\rm Gyr}}$
$\newcommand{\FeH}{{\rm[Fe/H]}}$
$\newcommand{\kmsd}{{\rm km/s/degree}}$
$\newcommand{\magg}{{\rm g}}$
$\newcommand{\magi}{{\rm i}}$
$\newcommand{\age}{{\rm age}}$
$\newcommand{\}{NFM}$
$\newcommand{\}{ADD}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\fmmm}[1]{\mbox{#1}}$
$\newcommand{\scnd}{\mbox{\fmmm{"}\hskip-0.3em .}}$
$\newcommand{\scnp}{\mbox{\fmmm{"}}}$
$\newcommand{\mcnd}{\mbox{\fmmm{'}\hskip-0.3em .}}$
$\newcommand{\ud}{\mathrm{d}}$
$\newcommand{\ltsima}{\; \buildrel < \over \sim \;}$
$\newcommand{\simlt}{\lower.5ex\hbox{\ltsima}}$
$\newcommand{\gtsima}{\; \buildrel > \over \sim \;}$
$\newcommand{\simgt}{\lower.5ex\hbox{\gtsima}}$
$\newcommand{\erf}{\mathop{\rm erf}\nolimits}$
$\newcommand{\sech}{ \mathop{\rm sech}\nolimits}$
$\newcommand{\csch}{ \mathop{\rm csch}\nolimits}$
$\newcommand{\arcsinh}{\mathop{\rm arcsinh}\nolimits}$
$\newcommand{\arccosh}{\mathop{\rm arccosh}\nolimits}$
$\newcommand{\arctanh}{\mathop{\rm arctanh}\nolimits}$
$\newcommand{\arccoth}{\mathop{\rm arccoth}\nolimits}$
$\newcommand{\arcsech}{\mathop{\rm arcsech}\nolimits}$
$\newcommand{\arccsch}{\mathop{\rm arccsch}\nolimits}$
$\newcommand{\arccot}{\mathop{\rm arccot}\nolimits}$
$\newcommand{\arcsec}{\mathop{\rm arcsec}\nolimits}$
$\newcommand{\arccsc}{\mathop{\rm arccsc}\nolimits}$
$\newcommand{\ylm}{\mathop{\rm Y}_l^m\nolimits}$
$\newcommand{\ylmp}{\mathop{\rm Y}_{l'}^{m'}\nolimits}$
$\newcommand{\real}{\Re e}$
$\newcommand{\imag}{\Im m}$
$\newcommand{\km}{{\rm km}}$
$\newcommand{\kms}{{\rm km s^{-1}}}$
$\newcommand{\kmsdeg}{{\rm km s^{-1} deg^{-1}}}$
$\newcommand{\kmskpc}{{\rm km s^{-1} kpc^{-1}}}$
$\newcommand{\mas}{{\rm mas}}$
$\newcommand{\masyr}{{\rm mas/yr}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand{\kpcdeg}{{\rm kpc deg^{-1}}}$
$\newcommand{\mpc}{{\rm Mpc}}$
$\newcommand{\msun}{{\rm M_\odot}}$
$\newcommand{\lsun}{{\rm L_\odot}}$
$\newcommand{\rsun}{{\rm R_\odot}}$
$\newcommand{\pc}{{\rm pc}}$
$\newcommand{\cm}{{\rm cm}}$
$\newcommand{\yr}{{\rm yr}}$
$\newcommand{\au}{{\rm AU}}$
$\newcommand{\g}{{\rm g}}$
$\newcommand{\om}{\Omega_0}$
$\newcommand{\}{ca}$
$\newcommand{\}{r}$
$\newcommand{\}{magnitude}$
$\newcommand{\kr}{{\cal K}_r}$
$\newcommand{\kz}{{\cal K}_z}$
$\newcommand{\kzz}{{\cal K}_z(z)}$
$\newcommand{\mss}{{\rm M}_\odot \rm pc^{-2}}$
$\newcommand{\msss}{{\rm M}_\odot \rm pc^{-3}}$
$\newcommand{\Aa}{\; \buildrel \circ \over{\rm A}}$
$\newcommand{Å}{\; \buildrel \circ \over{\rm A}}$
$\newcommand{\yr}{{\rm yr}}$
$\newcommand{\CompactFigs}{0}$
$\newcommand{\UseFigs}{1}$
$\newcommand{\deg}{^\circ}$
$\newcommand{\degg}{\hbox{\null^\circ\hskip-3pt .}}$
$\newcommand{\sec}{\hbox{"\hskip-3pt .}}$
$\newcommand{\half}{{\scriptstyle{1\over2}}}$
$\newcommand{\s}{\ifmmode \widetilde \else \~\fi}$
$\newcommand{\=}{\overline}$
$\newcommand{\scre}{{\cal E}}$
$\newcommand{\}{spose}$
$\newcommand{\larrow}{\leftarrow}$
$\newcommand{\rarrow}{\rightarrow}$
$\newcommand{\llangle}{\langle\langle}$
$\newcommand{\rrangle}{\rangle\rangle}$
$\newcommand{\etal}{{\it et al. }}$
$\newcommand{\cf}{{\it cf. }}$
$\newcommand{\eg}{{e.g., }}$
$\newcommand{\ie}{{ i.e., }}$
$\newcommand{\lta}{\mathrel{\spose{\lower 3pt\hbox{\mathchar"218}}$
$     \raise 2.0pt\hbox{\mathchar"13C}}}$
$\newcommand{\gta}{\mathrel{\spose{\lower 3pt\hbox{\mathchar"218}}$
$     \raise 2.0pt\hbox{\mathchar"13E}}}$
$\newcommand{\Dt}{\spose{\raise 1.5ex\hbox{\hskip3pt\mathchar"201}}}$
$\newcommand{\dt}{\spose{\raise 1.0ex\hbox{\hskip2pt\mathchar"201}}}$
$\newcommand{\del}{\nabla}$
$\newcommand{\delv}{\bb\nabla}$
$\newcommand{\r}{{\rm r^{1/4}}}$
$\newcommand{\jla}{J_{\lambda}}$
$\newcommand{\jmu}{J_{\mu}}$
$\newcommand{\jnu}{J_{\nu}}$
$\newcommand{\pomega}{\varpi}$
$\newcommand{\sigla}{\sigma_{\lambda}}$
$\newcommand{\sigmu}{\sigma_{\mu}}$
$\newcommand{\signu}{\sigma_{\nu}}$
$\newcommand{\dotsfill}{\leaders\hbox to 1em{\hss.\hss}\hfill}$
$\newcommand{\sun}{\odot}$
$\newcommand{\earth}{\oplus}$
$\newcommand{\Gyr}{{\rm Gyr}}$
$\newcommand{\FeH}{{\rm[Fe/H]}}$
$\newcommand{\kmsd}{{\rm km/s/degree}}$
$\newcommand{\magg}{{\rm g}}$
$\newcommand{\magi}{{\rm i}}$
$\newcommand{\age}{{\rm age}}$
$\newcommand{\}{NFM}$
$\newcommand{\}{ADD}$</div>



<div id="title">

# The PAndAS View of the Andromeda Satellite System. IV Global properties

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.01528-b31b1b.svg)](https://arxiv.org/abs/2303.01528)<mark>Appeared on: 2023-03-06</mark> - _Submitted to ApJ - 12 pages, 6 figures, 2 tables_

</div>
<div id="authors">

Amandine Doliva-Dolinsky, et al. -- incl., <mark><mark>Geraint F. Lewis</mark></mark>

</div>
<div id="abstract">

**Abstract:** We build a statistical framework to infer the global properties of the satellite system of the Andromeda galaxy (M31) from the properties of individual dwarf galaxies located in the Pan-Andromeda Archaelogical Survey (PAndAS) and the previously determined completeness of the survey. Using forward modeling, we infer the slope of the luminosity function of the satellite system, the slope of its spatial density distribution, and the size-luminosity relation followed by the dwarf galaxies. We find that the slope of the luminosity function is $\beta=-1.5\pm0.1$ . Combined with the spatial density profile, it implies that, when accounting for survey incompleteness, M31 hosts $92_{-26}^{+19}$ dwarf galaxies with $M_\textrm{V}<-5.5$ and a sky-projected distance from M31 between 30 and 300 \kpc . We conclude that many faint or distant dwarf galaxies remain to be discovered around Andromeda, especially outside the PAndAS footprint. Finally, we use our model to test if the higher number of satellites situated in the hemisphere facing the Milky Way could be explained simply by the detection limits of dwarf galaxy searches. We rule this out at $>99.9\%$ confidence and conclude that this anisotropy is an intrinsic feature of the M31 satellite system. The statistical framework we present here is a powerful tool to robustly constrain the properties of a satellite system and compare those across hosts, especially considering the upcoming start of the Euclid or Rubin large photometric surveys that are expected to uncover a large number of dwarf galaxies in the Local Volume.

</div>

<div id="div_fig1">

<img src="tmp_2303.01528/./function_lumi_3D.png" alt="Fig2" width="100%"/>

**Figure 2. -**  Cumulative number of dwarf galaxies as a function of their magnitude. The inferred model is represented by the black line and gray band. The dashed line shows the inference for the PAndAS footprint and the dot-dashed line the favored model once the average detection limits are applied. This line is directly comparable to, and shows good agreement with, the cumulative distribution known dwarf galaxies (teal points). (*luminosity_function_3D*)

</div>
<div id="div_fig2">

<img src="tmp_2303.01528/./mv_rh.png" alt="Fig1" width="100%"/>

**Figure 1. -**  Relation between the size and the luminosity of M31's dwarf galaxies as infer through our modeling. The best relation and corresponding width are represented by the purple full line and dashed lines, with the uncertainties on the mean model shown as the high opacity purple band. The best model derived by  ([Brasseur, et. al (2011)](https://ui.adsabs.harvard.edu/abs/2011ApJ...743..179B))  is represented by the orange lines, and the binary completeness limits they used by the dotted black line. The average detection limits folded in our analysis are represented by the grey background scale (100\% recovery in white and 0\% recovery in dark gray). Given those, the inferred model compensates for the undiscovered large and faint dwarf galaxies and therefore is slightly shifted from what we would naïvely expect from the cloud of known dwarf galaxies (teal dots). (*mv_rh_3D*)

</div>
<div id="div_fig3">

<img src="tmp_2303.01528/./DL_ani.png" alt="Fig6.1" width="50%"/><img src="tmp_2303.01528/./observed_density_ani.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** _Right panel_: Map of the completeness in the y-z plane with z being the distance to the MW. The contamination due to MW and M31 stars lead to the variation of the completeness along the y-axis, while the impact of the distance is visible along the z-axis. _Left panel_: Map of the observed galaxy density in the y-z plane. The red dots represent the known dwarf galaxies. The density is slightly asymmetric as it is somewhat higher on the positive end of the z-axis but this impact might be compensated by the increase of the observed space on the negative end. (*fig:anisotropy*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

218  publications in the last 7 days.
	 _build/html/2303.01528.md
	 _build/html/2303.00044.md
	 _build/html/2303.00012.md
	 _build/html/2302.14137.md
	 _build/html/2302.12805.md
	 _build/html/2302.10943.md
	 _build/html/2302.10528.md
	 _build/html/2302.10008.md
	 _build/html/2302.08962.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> daily </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.
